# Import

In [ ]:
%run init.ipynb

# Bitcoin

Idea: use the new price data to extend the current data

## Suply

In [ ]:
with open("../../temp/total-bitcoins.json") as f:
    total_btc = json.load(f)["total-bitcoins"]
total_btc[:3]

In [ ]:
total_btc = pd.DataFrame(total_btc)
total_btc.head(3)

In [ ]:
total_btc = pd.DataFrame({
    "ts":(total_btc["x"]/10**3).map(datetime.fromtimestamp),
    "n_coins":total_btc["y"]
}).set_index("ts").squeeze()
total_btc.head(3)

In [ ]:
total_btc = total_btc.resample("ME").last()
total_btc

In [ ]:
dump_pickle(total_btc, "total_btc")

## Volume

### The Block

In [ ]:
block_data = pd.read_csv(
    "../../temp/Volume_of_Bitcoin_Futures_(BTC_not_USD)_2025_12_18.csv",
    index_col="dt",
    parse_dates=True,
    date_format="%d/%m/%y %H:%M"
)
block_data.head(3)

In [ ]:
block_data = block_data.resample("ME")["volume"].sum()
block_data.head(3)

In [ ]:
dump_pickle(block_data, "btc_volume_block")

### CMEG

* Downloaded data manually
* Starts on 2013-01

In [ ]:
file_exists = pd.Series(
    False, 
    index=pd.date_range(start="2013-01-01", end="2025-08-01", freq="ME").strftime("%Y%m") + ".zip"
)
file_exists

In [ ]:
for i in file_exists.index.copy():
    file_exists.loc[i] = Path(f"../../temp/{i}").exists()
file_exists

In [ ]:
file_exists[file_exists == False]
# all files are present

In [ ]:
expected_files = list(
    "CMEG_" + pd.date_range(start="2013-01-01", end="2025-08-01", freq="ME").strftime("%Y%m") + ".pdf"
)
expected_files[:3]

In [ ]:
# ignore dot files
got_files = list(i.name for i in Path("../../temp/").iterdir() if not i.name.startswith("."))
got_files[:3]

In [ ]:
assert len(expected_files) == len(got_files), f"The number of actual files does not equal what is expected. Expected: {len(expected_files)}, got {len(got_files)}"

assert len(got_files) == len(set(got_files)), "There are duplicated in fetched files"

for exp, got in zip(sorted(expected_files), sorted(got_files)):
    assert exp == got, f"File names don't match: {exp} vs. {got}"
else:
    print(f"All expected files have been fetched and processed!")

In [ ]:
srs = pd.Series()

for file in Path("../../temp/").iterdir():
    if not file.name.startswith("CMEG"):
        continue

    for name in ["BITCOIN FUTURES", "BITCOIN"]:            
        try:
            volume = extract_asset_volume(
                path=file,
                asset=name
            )
            break
        except ValueError:
            volume = -1
    srs.loc[datetime(int(file.name[5:9]), int(file.name[9:11]), 1)] = volume
    print(f"{datetime(int(file.name[5:9]), int(file.name[9:11]), 1)}: {volume}")

srs = srs.sort_index()
srs[srs == -1] = np.nan
srs = srs * 5
srs

In [ ]:
dump_pickle(srs, "CMEG_volume")

## Concat

In [ ]:
block_volume = load_pickle("btc_volume_block")
block_volume.index.name = ""
block_volume

In [ ]:
cmeg_volume = load_pickle("CMEG_volume")
cmeg_volume = cmeg_volume.resample("ME").last()
cmeg_volume

In [ ]:
total_volume = (block_volume + cmeg_volume).dropna()
total_volume

In [ ]:
total_btc = load_pickle("total_btc")
total_btc

In [ ]:
df = pd.concat(
    [total_btc, total_volume],
    axis="columns",
    join="outer"
)
df = df.rename(columns={"n_coins":"supply", 0:"trading_volume"})
df = df.reindex(
    pd.date_range("2010-01-01", "2025-08-01", freq="ME")
)
df = df.dropna(axis="rows", how="any")
df

In [ ]:
df = df.reset_index()
df["index"] = "'" + df["index"].dt.strftime("%F") + "'"
df

In [ ]:
dry_insert_into_db(
    conn_str=CONNECTION_STRING, 
    table="bitcoin_trading_metadata",
    data=df.to_numpy()
)

In [ ]:
insert_into_db(
    conn_str=CONNECTION_STRING, 
    table="bitcoin_trading_metadata",
    data=df.to_numpy()
)

In [ ]:
logger.info("Bitcoin supply and demand have been acquired and inserted")

# S&P 500

## Supply

* downloaded data manually from the source

In [ ]:
df = pd.read_excel(
    io = "../../temp/snp_500_data.xlsx",
    sheet_name = "Historisch",
    header = 0,
    usecols = ["per", "Umlaufende Anteile"],
    na_values = "--",
    thousands = ",",
    decimal = "."
)
df.head(3)

In [ ]:
# split string dates into day, month, year
# some have form "08.Okt.2025", some are missing a full stop after the month "14.März2025"
# So we:
#     match the day into the first group, 
#     then skip the point, 
#     match month, 
#     optional point, 
#     match the year
pattern = re.compile(r"(\d{1,2})\.(Jan|Feb|März|Apr|Mai|Juni|Juli|Aug|Sept|Okt|Nov|Dez)\.?(\d{4})")
month_mapping = {k:v for k, v in zip("Jan|Feb|März|Apr|Mai|Juni|Juli|Aug|Sept|Okt|Nov|Dez".split("|"),range(1,13))}
dates = []

for i in df["per"]:
    match_object = re.match(pattern, i)
    group = list(match_object.groups())
    group[1] = month_mapping[group[1]]
    group = list(map(int, group))
    group = group[::-1]

    dates.append(
        datetime(*group)
    )
dates[:3]

In [ ]:
df["per"] = dates
df = df.rename(columns={"per":"dt", "Umlaufende Anteile": "outstanding_supply"})
df = df.set_index("dt")
df = df.squeeze().sort_index()
df

In [ ]:
# the supply does show significant downward fluctuations.
# Thus, monthly average will be taken
df.plot(kind="line")

In [ ]:
df = (df
    .resample("ME")
    .mean()
    .loc["2010-01-01":"2025-08-01"]
    .copy()
)
df

In [ ]:
df.plot(kind="line")

In [ ]:
dump_pickle(
    df,
    "snp_aggregated_supply_data"
)

## Volume

* Manually save the [source page](https://www.ishares.com/uk/individual/en/products/253743/ishares-sp-500-b-ucits-etf-acc-fund) into temp (HTML only)

In [ ]:
TICKER_TYPE = "Bloomberg Ticker"

with open("../../temp/iShares Core S&P 500 UCITS ETF _ CSPX.html") as f:
    html_dump = f.read()

soup = BeautifulSoup(html_dump, "html.parser")
table = soup.find("table",attrs={"id":"listingsTable"})
df = pd.read_html(
    StringIO(str(table)),
    na_values="-"
)[0]

del html_dump, soup, table

na_exchanges = list(df.loc[df[TICKER_TYPE].isna(), "Exchange"])
yf_tickers = list(df[TICKER_TYPE].dropna())

df

In [ ]:
yf_data = yf.download(
    tickers=list(df["RIC"].dropna()) + ["CSPX.AS", "CSSPX.SW"],
    interval="1mo",
    start="2010-01-01",
    end="2025-08-01",
    group_by="Ticker",
    keepna=True
)

yf_data = yf_data.xs(
    "Volume", 
    axis="columns", 
    level=1
)

trading_volume = yf_data.dropna(how="all").sum(axis="columns") / 0.93
trading_volume = trading_volume.resample("ME").last()
trading_volume

Missing exchanges:
* Bolsa De Valores De Colombia
* Santiago Stock Exchange
* Tel Aviv Stock Exchange

Together ~7% of the listed exchanges. Source: ChatGPT

In [ ]:
dump_pickle(trading_volume, "snp_trading_volume")

## Concat

In [ ]:
supply = load_pickle("snp_aggregated_supply_data")
volume = load_pickle("snp_trading_volume")

In [ ]:
df = pd.concat(
    [supply, volume],
    axis="columns",
    join="outer"
)
df = df.rename(columns={0:"volume"})

dates = pd.date_range(start="2010-01-01", end="2025-08-01", freq="ME")
df.reindex(dates).fillna(0)

In [ ]:
# validate index
pd.Series(df.index).is_monotonic_increasing

In [ ]:
df = df.reset_index()
df["index"] = "'" + df["index"].astype(str) + "'"
df

In [ ]:
dry_insert_into_db(
    CONNECTION_STRING,
    "snp_trading_metadata",
    df.to_numpy()
)

In [ ]:
insert_into_db(
    CONNECTION_STRING,
    "snp_trading_metadata",
    df.to_numpy()
)

In [ ]:
logger.info("S&P 500 supply and demand have been acquired and inserted")

# NASDAQ

## Supply

* downloaded data manually from the source

In [ ]:
df = pd.read_excel(
    io = "../../temp/nasdaq_data.xlsx",
    sheet_name = "Historical",
    header = 0,
    usecols = ["As Of", "Securities In Issue"],
    na_values = "--",
    thousands = ",",
    decimal = "."
)
df.head(3)

In [ ]:
# split string dates into day, month, year
# some have form "08.Okt.2025", some are missing a full stop after the month "14.März2025"
# So we:
#     match the day into the first group, 
#     then skip the point, 
#     match month, 
#     optional point, 
#     match the year
pattern = re.compile(r"(\d{1,2})\/(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sept|Oct|Nov|Dec)\/?(\d{4})")
month_mapping = {k:v for k, v in zip("Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sept|Oct|Nov|Dec".split("|"),range(1,13))}
dates = []

for i in df["As Of"]:
    match_object = re.match(pattern, i)
    group = list(match_object.groups())
    group[1] = month_mapping[group[1]]
    group = list(map(int, group))
    group = group[::-1]

    dates.append(
        datetime(*group)
    )
dates[:3]

In [ ]:
df["As Of"] = dates
df = df.rename(columns={"As Of":"dt", "Securities In Issue": "outstanding_supply"})
df = df.set_index("dt")
df = df.squeeze().sort_index()
df

In [ ]:
# the supply does show significant downward fluctuations.
# Thus, monthly average will be taken
df.plot(kind="line")

In [ ]:
df = (df
    .resample("ME")
    .mean()
    .loc["2010-01-01":"2025-08-01"]
    .copy()
)
df

In [ ]:
df.plot(kind="line")

In [ ]:
dump_pickle(
    df,
    "nasdaq_aggregated_supply_data"
)

## Volume

In [ ]:
TICKER_TYPE = "Bloomberg Ticker"

with open("../../temp/iShares NASDAQ 100 UCITS ETF _ CNDX.html") as f:
    html_dump = f.read()

soup = BeautifulSoup(html_dump, "html.parser")
table = soup.find("table",attrs={"id":"listingsTable"})
df = pd.read_html(
    StringIO(str(table)),
    na_values="-"
)[0]

del html_dump, soup, table

na_exchanges = list(df.loc[df[TICKER_TYPE].isna(), "Exchange"])
yf_tickers = list(df[TICKER_TYPE].dropna())

df

* Tel Aviv is missing

In [ ]:
yf_data = yf.download(
    tickers=list(df["RIC"].dropna()) + ["CNDX.AS", "CSNDX.SW"],
    interval="1mo",
    start="2010-01-01",
    end="2025-08-01",
    group_by="Ticker",
    keepna=True
)

yf_data = yf_data.xs(
    "Volume", 
    axis="columns", 
    level=1
)

trading_volume = yf_data.dropna(how="all").sum(axis="columns") / 0.93
trading_volume = trading_volume.resample("ME").last()
trading_volume

In [ ]:
dump_pickle(
    trading_volume,
    "nasdaq_trading_volume"
)

## Concat

In [ ]:
supply = load_pickle("nasdaq_aggregated_supply_data")
trading_volume = load_pickle("nasdaq_trading_volume")

In [ ]:
supply

In [ ]:
trading_volume

In [ ]:
df = pd.concat(
    [supply, trading_volume],
    axis="columns",
    join="outer"
)
df = df.rename(columns={0:"trading_volume"})
df.head(3)

In [ ]:
df = df.reset_index()
df["index"] = "'" + df["index"].astype("str") + "'"
df

In [ ]:
dry_insert_into_db(
    CONNECTION_STRING,
    "nasdaq_trading_metadata",
    df.to_numpy()
)

In [ ]:
insert_into_db(
    CONNECTION_STRING,
    "nasdaq_trading_metadata",
    df.to_numpy()
)

In [ ]:
logger.info("Successfully inserted NASDAQ trading metadata")

# Dow Jones